In [226]:
import json
import pandas as pd
import numpy as np
import pickle
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter
from time import time

In [227]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)

In [228]:
pickle_in = open(r"C:\Users\surya\Desktop\Jupyter Python Programs\Amazon Review NLP\Review_features_data.pickle", 'rb')
df = pickle.load(pickle_in)
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4,22,21,228544.0
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1,10,9,228544.0
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,22,21,228544.0
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,5,5,228544.0
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,12,12,228544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041164,5,True,"10 3, 2016",A3Q7GA2Q62E18Q,B01HE0RNYO,NaN,Kerry Moore,Excellent product for the price!!!,Inexpensive but good quality...,1475452800,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,3,3,574746.0
1041165,5,False,"07 18, 2016",AEKOIX49JGCUG,B01HE0RNYO,NaN,Wishart,I needed a new phone case for my iPhone 6 plus...,Great Case. Gonna Buy Again.,1468800000,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,24,152,114,574746.0
1041166,4,False,"07 16, 2016",A2KOKSKEDIBRGQ,B01HE0RNYO,NaN,Wise One,The covers my iPhone just fine. It is simple i...,Plain Jane of iPhone Covers,1468627200,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,28,233,159,574746.0
1041167,3,False,"07 11, 2016",A1BO3NH4KKASUT,B01HE0RNYO,NaN,Bebescastle,This free case is okay it sli

In [229]:
rows = list(df.index)
rows

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


## Getting rank of 2nd element in 'rank' list and making subcategories

In [230]:
a = df.iloc[0]['rank']
a

"['>#228,544 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']"

In [231]:
# Ref - https://stackoverflow.com/questions/24890779/python-list-inside-string-to-list
a = eval(a)
a

['>#228,544 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)',
 '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']

In [232]:
a = a[1]
a

'>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips'

In [233]:
def rank_calc(rank_str):
    rank = ''
    for i in range(len(rank_str)):
        if rank_str[i].isdigit() == True:
            rank += rank_str[i]
        elif rank_str[i] == 'i':
            break
    return rank

In [234]:
rank_calc(a)

'114920'

In [235]:
rank_list1 = [rank_calc(eval(df.iloc[i]['rank'])[1]) for i in range(343534,343544)]
rank_list1

['33078',
 '33078',
 '33078',
 '142956',
 '142956',
 '142956',
 '142956',
 '142956',
 '142956',
 '142956']

### In rank column, some rows do not follow the usual format (they are starting with 111,719 having no follow up rank and subcategories), so removing them.

In [236]:
for i in rows:
    if '111,719' in df.iloc[i]['rank']:
        print(i)

667
668
669
670
671
672
673
674
675
676
677
678


In [237]:
for i in range(667,680):
    print(df.iloc[i]['rank'])

111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
111,719 in Cell Phones & Accessories (
['>#3,059,436 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#4,085 in Cell Phones & Accessories > Carrier Cell Phones']


The last one is how all rows should be formatted.

In [238]:
df.drop(list(range(667,679)), inplace=True)

In [239]:
df.reset_index(drop=True, inplace=True)
rows = list(df.index)

### Some rank rows have nothing in them (i.e. []), which makes it impossible to run the code which takes the 1st index (2nd element) of the list, as there is nothing there, so dropping them.

In [240]:
df[df['rank'] == '[]']

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no
530,4,False,"01 22, 2005",A1TTKKN4AY63QK,B00005UK9X,{'Package Type:': ' Standard Packaging'},Jan R. Klincewicz,"I drive a company-supplied vehicle, so alterat...",Rocks for normal drivers,1106352000,26,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,62,54,NaN
531,5,True,"06 25, 2003",A16PKR0PIB6959,B00005UK9X,{'Package Type:': ' Standard Packaging'},David Levine,I have a GPS V that I really like and this is ...,Great add-on,1056499200,14,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,62,52,NaN
532,5,True,"11 11, 2012",A1V4KTY1RXI606,B00005UK9X,{'Package Type:': ' Standard Packaging'},Barbara P.,As long as there are products like this availa...,A real grabber,1352592000,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,20,19,NaN
533,5,True,"08 31, 2009",A19RGDJPNZ7NZ,B00005UK9X,{'Package Type:': ' Standard Packaging'},MW,"I use it with a Garmin StreetPilot 7200, which...",Works with Garmin 7200,1251676800,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,82,71,NaN
534,5,True,"04 4, 2008",A100WO06OQR8BQ,B00005UK9X,{'Package Type:': ' Standard Packaging'},Duffer.,Works great. Easy to attach the GPS and easy t...,Great Device,1207267200,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4,13,12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041142,1,True,"08 11, 2016",AKJK5FH23L8ZA,B01H72UJZ4,NaN,Rodolfo Tarelli,Over one milimiter uncovered in all the temper...,Really Bad Fit,1470873600,0,['https://images-na.ssl-images-amazon.com/imag...,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,5,22,20,NaN
1041143,2,False,"08 3, 2016",A3TZH1A8PYUK1O,B01H72UJZ4,NaN,TuckerReviewer,"This is a clear glass protector, theoretically...",Not sized correctly.,1470182400,2,['https://images-na.ssl-images-amazon.com/imag...,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,9,57,47,NaN
1041144,4,False,"08 2, 2016",ARKQFG8F2FJ8L,B01H72UJZ4,NaN,Amazon Customer,I've used quite a few screen protectors over y...,"Durable, a little narrow",1470096000,0,NaN,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,19,137,108,NaN
1041145,3,False,"07 20, 2016",A24BH8L76QQFVD,B01H72UJZ4,NaN,PastorONeil,This is a tempered glass screen protector for ...,This is a bit annoying as dust and other parti...,1468972800,0,NaN,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,17,157,101,NaN


In [241]:
nothing_list = [i for i in rows if df.iloc[i]['rank'] == '[]']

In [242]:
nothing_list

[530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 573,
 574,
 575,
 576,
 577,
 578,
 579,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 587,
 588,
 589,
 590,
 591,
 592,
 593,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 699,
 700,
 701,
 702,
 703,
 704,
 705,
 706,
 707,
 708,
 709,
 710,
 711,
 712,
 713,
 714,
 715,
 716,
 717,
 718,
 719,
 720,
 721,
 722,
 723,
 724,
 725,
 726,
 727,
 728,
 729,
 730,
 731,
 732,
 733,
 734,
 735,
 736,
 737,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 746,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 755,
 756,
 757,
 758,
 759,
 760,
 761,
 762,
 763,
 764,
 765,
 766,
 767,
 768,
 769,
 770,
 771,
 772,
 773,
 774,
 775,
 776,
 777,
 778,
 779

In [243]:
len(nothing_list)

66796

In [244]:
df.drop(nothing_list, inplace=True)

In [245]:
df.reset_index(drop=True, inplace=True)
rows = list(df.index)

### In rank column, some rows do not follow the usual format (they are starting with 1,420,411 having no follow up rank and subcategories), so removing them.

In [246]:
smlist = [i for i in rows if '1,420,411' in df.iloc[i]['rank']]

In [247]:
smlist

[1596, 1597, 1598, 1599, 1600, 1601]

In [248]:
for i in smlist:
    print(df.iloc[i]['rank'])

1,420,411 in Cell Phones & Accessories (
1,420,411 in Cell Phones & Accessories (
1,420,411 in Cell Phones & Accessories (
1,420,411 in Cell Phones & Accessories (
1,420,411 in Cell Phones & Accessories (
1,420,411 in Cell Phones & Accessories (


In [250]:
df.drop(smlist, inplace=True)

In [251]:
df.reset_index(drop=True, inplace=True)
rows = list(df.index)

### In rank column, some rows do not follow the usual format (they are starting with 1,618,849 having no follow up rank and subcategories), so removing them.

In [256]:
smlist1 = [i for i in rows if '1,618,849' in df.iloc[i]['rank']]

In [257]:
smlist1

[3291, 3292, 3293, 3294, 3295, 3296]

In [258]:
for i in smlist1:
    print(df.iloc[i]['rank'])

1,618,849 in Cell Phones & Accessories (
1,618,849 in Cell Phones & Accessories (
1,618,849 in Cell Phones & Accessories (
1,618,849 in Cell Phones & Accessories (
1,618,849 in Cell Phones & Accessories (
1,618,849 in Cell Phones & Accessories (


In [259]:
df.drop(smlist1, inplace=True)

In [260]:
df.reset_index(drop=True, inplace=True)
rows = list(df.index)

In [ ]:
2+2

In [282]:
drop_list = [i for i in rows if df.iloc[i]['rank'][0].isdigit() == True]

In [283]:
drop_list

[7548,
 7549,
 7550,
 7551,
 7552,
 7553,
 7554,
 7782,
 7783,
 7784,
 7785,
 7786,
 7787,
 7788,
 7789,
 7790,
 7791,
 7792,
 7793,
 7794,
 7795,
 11066,
 11067,
 11068,
 11069,
 11070,
 11071,
 11072,
 11073,
 11074,
 11075,
 11076,
 11077,
 11078,
 11079,
 11080,
 13078,
 13079,
 13080,
 13081,
 13082,
 13083,
 13084,
 13085,
 13086,
 13087,
 13088,
 13089,
 13090,
 13091,
 13092,
 13093,
 13094,
 13095,
 16551,
 16552,
 16553,
 16554,
 16555,
 16556,
 16557,
 19147,
 19148,
 19149,
 19150,
 19151,
 19152,
 21578,
 21579,
 21580,
 21581,
 21582,
 21583,
 21584,
 21585,
 23511,
 23512,
 23513,
 23514,
 23515,
 23516,
 23517,
 23518,
 23580,
 23581,
 23582,
 23583,
 23584,
 23585,
 23586,
 23587,
 23588,
 23589,
 23590,
 23591,
 23592,
 23593,
 23920,
 23921,
 23922,
 23923,
 23924,
 23925,
 23926,
 23927,
 23928,
 23929,
 23930,
 23931,
 23932,
 23933,
 23934,
 23935,
 24266,
 24267,
 24268,
 24269,
 24270,
 24271,
 24272,
 24273,
 24274,
 24275,
 24276,
 24277,
 24278,
 24279,
 2428

In [284]:
len(drop_list)

9268

In [290]:
df.drop(drop_list, inplace=True)

In [291]:
df.reset_index(drop=True, inplace=True)
rows = list(df.index)

In [293]:
start = time()

In [294]:
rank_list = [rank_calc(eval(df.iloc[i]['rank'])[1]) for i in rows]

IndexError: list index out of range

In [ ]:
end = time()
end - start

In [ ]:
rank_list